In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

## Download data for the model:

In [ ]:
# Define paramters for data donload: 
ticker = 'SPOT'
api_key = 'da2d62a4a8a1529c5f4b4ff4341089ce'

# Get Income Statement data: 
IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{ticker}?apikey={api_key}').json()

BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={api_key}').json()

comp_profile = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}').json()


## define functions for the model build: 


In [ ]:
# Define function to create DF to hold the model: 
def df_creator(base_year, forecast_period): 
  col_names = [str(base_year)+'A']
  for i in range(1, forecast_period+1):
    year = base_year +i
    col_names.append(str(year)+'F')
  col_names.append('TV')
  
  row_names = ['Revenue growth rate', 'Revenue', 'EBIT margin', 'EBIT', 'Tax Rate', 'EBIT(1 - t)', 'Reinvestment', 'FCFF',
               'WACC', 'DF', 'PV_FCFF', 'Sales to capital ratio', 'Invested capital', 'ROIC']

  return pd.DataFrame(index=row_names, columns=col_names)


def TV_creator():
  col_name = ['Terminal Value']
  row_names = ['Terminal CF', 'Terminal cost of capital' , 'TV', 'PV of TV', 'PV of CFs', 'Sum of PV', 'Prob of failure', 
               'Proceeds if failure', 'Value of Op assets', 'Debt', 'Cash', 'Value of equity', 'number of sh out', 'Estimated px']
  
  return pd.DataFrame(index = row_names, columns = col_name)
               



def revenue_growth_forecast(forecast_period, initial_assumption, convergence_period, tv_assumption):
  revenue_ff = np.ones(forecast_period+2)*initial_assumption
  convergence_rate = (initial_assumption-tv_assumption)/(forecast_period - convergence_period)
  temp= []
  for i in range(1,convergence_period+1):
    temp.append(initial_assumption - convergence_rate*i)
  
  revenue_ff[convergence_period+1: forecast_period+1] = temp
  revenue_ff[-1] = tv_assumption

  revenue_ff[0] = (IS[0]['revenue'] - IS[1]['revenue'])/IS[0]['revenue']
  
  return revenue_ff


def historical_values(tax_assumption):
  revenue_hist = IS[0]['revenue'] 
  ebit_op_inc = IS[0]['operatingIncome']
  ebit_op_margin = ebit_op_inc/revenue_hist
  
  if ebit_op_inc*(1 - tax_assumption)<0:
    ebit_after_tax = 0
  else:
    ebit_after_tax = ebit_op_inc*(1 - tax_assumption)
  
  return revenue_hist, ebit_op_inc, ebit_op_margin, ebit_after_tax


def revenue_forecast(DCF_model, forecast_period):
  for i in range(1,forecast_period+2):
    DCF_model.loc['Revenue'][i] = DCF_model.loc['Revenue'][i-1]*(1 + DCF_model.loc['Revenue growth rate'][i])
  return DCF_model


def assumption_forecast_ebit_margin(initial_estimate, convergence_period, tv_assumption, forecast_period):
  convergence_rate = (tv_assumption - initial_estimate)/convergence_period

  ebit_margin = np.ones(forecast_period+2)
  ebit_margin = ebit_margin*tv_assumption

  temp = []
  for i in range(0,convergence_period):
    temp.append(initial_estimate + convergence_rate*i)

  ebit_margin[1:convergence_period+1] = temp  

  # define historical value: 

  revenue_hist = IS[0]['revenue'] 
  ebit_op_inc = IS[0]['operatingIncome']
  ebit_op_margin = ebit_op_inc/revenue_hist

  ebit_margin[0] = ebit_op_margin

  return ebit_margin


def tax_rate_assumption(tax_rate, forecast_period):
  tax_rate_assumption = np.ones(forecast_period+2)*tax_rate
  return tax_rate_assumption

def sales_to_capital_assumption(first_stage, second_stage, forecasting_period, first_stage_length):
  sales_to_capital_assumption = np.ones(forecasting_period+1)*first_stage
  sales_to_capital_assumption[first_stage_length+1:] = second_stage

  return sales_to_capital_assumption


def beta_calculation():
  pass


def cost_of_equity(rf, ERP): 
  beta = comp_profile[0]['beta']
  return rf + beta*ERP

def cost_of_capital(cost_of_equity, cost_of_debt, tax_rate):
  total_debt = BS[0]['totalDebt']
  equity = BS[0]['totalStockholdersEquity']
  debt_weight = total_debt/(total_debt + equity)
  equity_weight = equity/(total_debt+equity)

  return (cost_of_equity*equity_weight + cost_of_debt*debt_weight*(1-tax_rate))




## Model build up

In [ ]:
# Create DataFrame object to hold the model: 
SPOT_model = df_creator(2020, 10)
SPOT_tv = TV_creator()

In [ ]:
# Define revenue growth forecast and input these estimates into the model object: 
revenue_forecast_array = revenue_growth_forecast(10, 0.23, 5, 0.045)
SPOT_model.loc['Revenue growth rate'] = revenue_forecast_array

In [ ]:
# Determine and assign other historical values to the model: 
revenue_hist, ebit_op_inc, ebit_op_margin, ebit_after_tax = historical_values(0.25)
SPOT_model.loc['Revenue']['2020A'] = revenue_hist


In [ ]:
# Forecast company's revenue based on the forecasted growth rate:
SPOT_model = revenue_forecast(SPOT_model, 10)

In [ ]:
# Forecast operating income margins:
operating_margins=assumption_forecast_ebit_margin(0.08, 8, 0.18, 10)
SPOT_model.loc['EBIT margin'] = operating_margins

In [ ]:
# Calculate operating_income (EBIT):
SPOT_model.loc['EBIT'] = SPOT_model.loc['Revenue']*SPOT_model.loc['EBIT margin']

In [ ]:
# Insert tax rate assumption into the model object: 
tax_rate_def = tax_rate_assumption(0.25, 10)
SPOT_model.loc['Tax Rate'] = tax_rate_def

In [ ]:
# Calculate EBIT(1 - t) based on the assumptions: 
SPOT_model.loc['EBIT(1 - t)'] = SPOT_model.loc['EBIT']*SPOT_model.loc['Tax Rate']

In [ ]:
# Define sales to capital ratio assumption: 
stca = sales_to_capital_assumption(20.0, 20.0, 10, 7)
SPOT_model.loc['Sales to capital ratio'][1:] = stca 

In [ ]:
# Calculate reinvestment amount: 
temp = [0]
for i in range(11):
  temp.append((SPOT_model.loc['Revenue'][i+1] - SPOT_model.loc['Revenue'][i])/SPOT_model.loc['Sales to capital ratio'][i+1])

SPOT_model.loc['Reinvestment'] = temp

In [ ]:
# Calculate the FCFF: 
SPOT_model.loc['FCFF'] = SPOT_model.loc['EBIT(1 - t)'] - SPOT_model.loc['Reinvestment']

In [ ]:
# calculate cost of capital: 
ce = cost_of_equity(0.01, 0.052)
wacc = cost_of_capital(ce, 0.05, 0.25)

# Assign the wacc variable to the appropriate row in the DF: 
SPOT_model.loc['WACC'][1:] = wacc

In [ ]:
# Calculate the discount factor for FCFF: 
temp = [1]
for i in range(1,11):
  temp.append(temp[i-1]*(1/(1 + SPOT_model.loc['WACC'][i])))

# Assign the calculated values to the DF: 
SPOT_model.loc['DF'][1:11] = temp[1:]

In [ ]:
# Calculate the PF of FCFF: 
SPOT_model.loc['PV_FCFF'][1:11] = SPOT_model.loc['DF'][1:11]*SPOT_model.loc['FCFF'][1:11]

In [ ]:
# SPOT Terminal Value: 

SPOT_tv.loc['Terminal CF']['Terminal Value'] = SPOT_model.loc['FCFF'][-1]

In [ ]:
SPOT_tv.loc['Terminal cost of capital'] = wacc

In [ ]:
# Calculate TV: 

SPOT_tv.loc['TV'] = SPOT_tv.loc['Terminal CF']/(SPOT_tv.loc['Terminal cost of capital'] - SPOT_model.loc['Revenue growth rate'][-1])

In [ ]:
# Calculate PV of TV: 

SPOT_tv.loc['PV of TV'] = SPOT_tv.loc['TV']*SPOT_model.loc['DF'][-2]

In [ ]:
SPOT_tv.loc['PV of CFs'] = SPOT_model.loc['PV_FCFF'].sum()

In [ ]:
SPOT_tv.loc['Sum of PV'] = SPOT_tv.loc['PV of TV'] + SPOT_tv.loc['PV of CFs']

In [ ]:
# Define probability of failure for SPOT: 
SPOT_tv.loc['Prob of failure'] = 0.1

In [ ]:
# Calculate Proceeds if failure occurs (for simplicty assume it to be half of sum of PV): 

SPOT_tv.loc['Proceeds if failure'] = 0.5*SPOT_tv.loc['Sum of PV']

In [ ]:
# Calculate value of operating assets (prob weighted average of assets: success vs failure): 
SPOT_tv.loc['Value of Op assets'] = (1 - SPOT_tv.loc['Prob of failure'] )*SPOT_tv.loc['Sum of PV'] + SPOT_tv.loc['Prob of failure']*SPOT_tv.loc['Proceeds if failure']

In [ ]:
SPOT_tv.loc['Debt'] = BS[0]['totalDebt']
SPOT_tv.loc['Cash'] = BS[0]['cashAndCashEquivalents']

In [ ]:
# Calculate value of equity: 

SPOT_tv.loc['Value of equity'] = SPOT_tv.loc['Value of Op assets'] - SPOT_tv.loc['Debt'] + SPOT_tv.loc['Cash']

In [ ]:
# Assign number of shares outstanding to the DF: 
SPOT_tv.loc['number of sh out'] = IS[0]['weightedAverageShsOutDil']

In [ ]:

SPOT_tv.loc['Estimated px'] = SPOT_tv.loc['Value of equity']/SPOT_tv.loc['number of sh out']

## Model check: 

In [ ]:
SPOT_model

,2020A,2021F,2022F,2023F,2024F,2025F,2026F,2027F,2028F,2029F,2030F,TV
Revenue growth rate,0.141624,0.23,0.23,0.23,0.23,0.23,0.193,0.156,0.119,0.082,0.045,0.045
Revenue,7880000000,9.6924e+09,1.19217e+10,1.46636e+10,1.80363e+10,2.21846e+10,2.64662e+10,3.0595e+10,3.42358e+10,3.70431e+10,3.871e+10,4.0452e+10
EBIT margin,-0.0371827,0.08,0.0925,0.105,0.1175,0.13,0.1425,0.155,0.1675,0.18,0.18,0.18
EBIT,-2.93e+08,7.75392e+08,1.10275e+09,1.53968e+09,2.11926e+09,2.884e+09,3.77144e+09,4.74222e+09,5.73449e+09,6.66776e+09,6.96781e+09,7.28136e+09
Tax Rate,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25
EBIT(1 - t),-7.325e+07,1.93848e+08,2.75688e+08,3.8492e+08,5.29815e+08,7.21e+08,9.4286e+08,1.18556e+09,1.43362e+09,1.66694e+09,1.74195e+09,1.82034e+09
Reinvestment,0,9.062e+07,1.11463e+08,1.37099e+08,1.68632e+08,2.07417e+08,2.14081e+08,2.06437e+08,1.8204e+08,1.40367e+08,8.3347e+07,8.70976e+07
FCFF,-7.325e+07,1.03228e+08,1.64226e+08,2.47821e+08,3.61184e+08,5.13583e+08,7.28778e+08,9.79118e+08,1.25158e+09,1.52657e+09,1.65861e+09,1.73324e+09
WACC,NaN,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894,0.0801894
DF,NaN,0.925764,0.857038,0.793415,0.734514,0.679987,0.629507,0.582775,0.539511,0.49946,0.462382,NaN


In [ ]:
SPOT_tv

,Terminal Value
Terminal CF,1.73324e+09
Terminal cost of capital,0.0801894
TV,4.92547e+10
PV of TV,2.27745e+10
PV of CFs,4.28145e+09
Sum of PV,2.70559e+10
Prob of failure,0.1
Proceeds if failure,1.3528e+10
Value of Op assets,2.57031e+10
Debt,577000000
